# 09 - Story Generation & Metadata Renderers

## 🧭 Goal

Understand how ODIBI tracks pipeline execution metadata and generates stories in multiple formats.

This notebook will:
- Show how ODIBI captures pipeline execution metadata automatically
- Demonstrate PipelineStoryMetadata and NodeExecutionMetadata
- Run a small pipeline and capture metadata
- Use HTMLStoryRenderer, MarkdownStoryRenderer, JSONStoryRenderer
- Generate stories in all 3 formats (HTML, Markdown, JSON)
- Show what gets tracked: duration, rows in/out, schema changes, status

**Estimated time:** 30 seconds

---

## 🧱 Core Concepts

**Pipeline Metadata Tracking:**
```python
# ODIBI automatically tracks execution details
result = pipeline.run()
metadata = result.metadata  # Contains all execution info

# Render stories in multiple formats
HTMLStoryRenderer().render_to_file(metadata, 'story.html')
MarkdownStoryRenderer().render_to_file(metadata, 'story.md')
JSONStoryRenderer().render_to_file(metadata, 'story.json')
```

**What Gets Tracked?**
- ⏱️ Duration (total and per-node)
- 📊 Rows in/out and change percentages
- 📋 Schema changes (columns added/removed)
- ✅ Status (success/failed/skipped)
- 📝 Operation names and parameters
- 🕐 Timestamps (start/end)


## 🔧 Setup

In [ ]:
# ✅ Environment Setup
import os
from pathlib import Path
import pandas as pd
import json
from datetime import datetime

# Navigate to project root
project_root = Path.cwd().parent if Path.cwd().name == "walkthroughs" else Path.cwd()
os.chdir(project_root)

# Create artifacts directory
artifacts_dir = Path("walkthroughs/.artifacts/09_story")
artifacts_dir.mkdir(parents=True, exist_ok=True)

# Import ODIBI story components
from odibi.story import (
    PipelineStoryMetadata,
    NodeExecutionMetadata,
    HTMLStoryRenderer,
    MarkdownStoryRenderer,
    JSONStoryRenderer,
)

print("✅ Environment ready")
print(f"📁 Artifacts: {artifacts_dir}")

## ▶️ Run: Create and Execute a 3-Node Pipeline

In [ ]:
import time

# 🧪 Simulate a 3-node pipeline execution with metadata tracking
print("🚀 Running 3-node pipeline...\n")

# Create pipeline metadata container
pipeline_metadata = PipelineStoryMetadata(
    pipeline_name="demo_data_transformation",
    pipeline_layer="bronze_to_silver",
    project="ODIBI",
    plant="Demo Plant",
    asset="Sensor Data",
)

start_time = time.time()

# Node 1: Load Data
node1_start = time.time()
df1 = pd.DataFrame(
    {
        "sensor_id": ["S001", "S002", "S003", "S004", "S005"],
        "temperature": [22.5, 23.1, 21.8, 24.2, 22.9],
        "humidity": [45, 48, 43, 52, 46],
        "timestamp": pd.date_range("2024-01-01", periods=5, freq="h"),
    }
)
node1_duration = time.time() - node1_start

node1_metadata = NodeExecutionMetadata(
    node_name="load_sensor_data",
    operation="read_csv",
    status="success",
    duration=node1_duration,
    rows_in=0,
    rows_out=len(df1),
    schema_in=[],
    schema_out=list(df1.columns),
    started_at=datetime.now().isoformat(),
    completed_at=datetime.now().isoformat(),
)
node1_metadata.calculate_row_change()
node1_metadata.calculate_schema_changes()
pipeline_metadata.add_node(node1_metadata)

print("✅ Node 1: load_sensor_data")
print(f"   Rows out: {len(df1)}")
print(f"   Schema: {list(df1.columns)}")
print(f"   Duration: {node1_duration:.4f}s\n")

# Node 2: Add Calculated Column
node2_start = time.time()
df2 = df1.copy()
df2["heat_index"] = df2["temperature"] * 1.1 + df2["humidity"] * 0.05
node2_duration = time.time() - node2_start

node2_metadata = NodeExecutionMetadata(
    node_name="calculate_heat_index",
    operation="add_column",
    status="success",
    duration=node2_duration,
    rows_in=len(df1),
    rows_out=len(df2),
    schema_in=list(df1.columns),
    schema_out=list(df2.columns),
    started_at=datetime.now().isoformat(),
    completed_at=datetime.now().isoformat(),
)
node2_metadata.calculate_row_change()
node2_metadata.calculate_schema_changes()
pipeline_metadata.add_node(node2_metadata)

print("✅ Node 2: calculate_heat_index")
print(f"   Rows in: {len(df1)} → Rows out: {len(df2)}")
print(f"   Columns added: {node2_metadata.columns_added}")
print(f"   Duration: {node2_duration:.4f}s\n")

# Node 3: Filter High Temperature
node3_start = time.time()
df3 = df2[df2["temperature"] > 22.0].copy()
node3_duration = time.time() - node3_start

node3_metadata = NodeExecutionMetadata(
    node_name="filter_high_temp",
    operation="filter",
    status="success",
    duration=node3_duration,
    rows_in=len(df2),
    rows_out=len(df3),
    schema_in=list(df2.columns),
    schema_out=list(df3.columns),
    started_at=datetime.now().isoformat(),
    completed_at=datetime.now().isoformat(),
)
node3_metadata.calculate_row_change()
node3_metadata.calculate_schema_changes()
pipeline_metadata.add_node(node3_metadata)

print("✅ Node 3: filter_high_temp")
print(f"   Rows in: {len(df2)} → Rows out: {len(df3)}")
print(f"   Row change: {node3_metadata.rows_change} ({node3_metadata.rows_change_pct:+.1f}%)")
print(f"   Duration: {node3_duration:.4f}s\n")

# Complete pipeline metadata
total_duration = time.time() - start_time
pipeline_metadata.duration = total_duration
pipeline_metadata.completed_at = datetime.now().isoformat()

print("🎉 Pipeline completed!")
print(f"   Total nodes: {pipeline_metadata.total_nodes}")
print(f"   Completed: {pipeline_metadata.completed_nodes}")
print(f"   Success rate: {pipeline_metadata.get_success_rate():.1f}%")
print(f"   Total duration: {total_duration:.4f}s")

## 🔍 Inspect: Examine Captured Metadata

In [ ]:
# Inspect pipeline-level metadata
print("📊 Pipeline Metadata:\n")
print(f"Name: {pipeline_metadata.pipeline_name}")
print(f"Layer: {pipeline_metadata.pipeline_layer}")
print(f"Duration: {pipeline_metadata.duration:.4f}s")
print(f"Total nodes: {pipeline_metadata.total_nodes}")
print(f"Completed: {pipeline_metadata.completed_nodes}")
print(f"Failed: {pipeline_metadata.failed_nodes}")
print(f"Success rate: {pipeline_metadata.get_success_rate():.1f}%")
print(f"Total rows processed: {pipeline_metadata.get_total_rows_processed()}\n")

# Inspect node-level metadata
print("📋 Node Execution Details:\n")
for node in pipeline_metadata.nodes:
    print(f"  🔹 {node.node_name}")
    print(f"     Operation: {node.operation}")
    print(f"     Status: {node.status}")
    print(f"     Duration: {node.duration:.4f}s")
    print(f"     Rows: {node.rows_in} → {node.rows_out}")
    if node.rows_change is not None:
        print(f"     Change: {node.rows_change:+d} ({node.rows_change_pct:+.1f}%)")
    if node.columns_added:
        print(f"     Columns added: {node.columns_added}")
    if node.columns_removed:
        print(f"     Columns removed: {node.columns_removed}")
    print()

# Save metadata as JSON for inspection
with open(artifacts_dir / "pipeline_metadata.json", "w") as f:
    json.dump(pipeline_metadata.to_dict(), f, indent=2)

print(f"✅ Metadata exported to: {artifacts_dir / 'pipeline_metadata.json'}")

## 🎨 Create: Generate Stories in All 3 Formats

In [ ]:
# Initialize renderers
html_renderer = HTMLStoryRenderer()
markdown_renderer = MarkdownStoryRenderer()
json_renderer = JSONStoryRenderer()

print("📝 Generating stories in multiple formats...\n")

# Generate HTML Story
html_path = html_renderer.render_to_file(pipeline_metadata, artifacts_dir / "story.html")
print(f"✅ HTML story: {html_path}")
print("   Open in browser to view interactive report")

# Generate Markdown Story
markdown_path = markdown_renderer.render_to_file(pipeline_metadata, artifacts_dir / "story.md")
print(f"✅ Markdown story: {markdown_path}")
print("   Human-readable documentation")

# Generate JSON Story
json_path = json_renderer.render_to_file(pipeline_metadata, artifacts_dir / "story.json")
print(f"✅ JSON story: {json_path}")
print("   Machine-readable for API integration")

print("\n🎉 All 3 story formats generated successfully!")

## 📖 View: Preview Markdown Story

In [ ]:
# Preview the markdown story
with open(artifacts_dir / "story.md", "r") as f:
    markdown_content = f.read()

print("📖 Markdown Story Preview:\n")
print("=" * 80)
print(markdown_content[:1000])  # Show first 1000 characters
print("...\n")
print("=" * 80)
print(f"\nℹ️  Full content: {len(markdown_content)} characters")
print(f"📄 Read complete story at: {artifacts_dir / 'story.md'}")

## 🔬 Analyze: What Was Tracked?

In [ ]:
# Load and analyze the JSON story
with open(artifacts_dir / "story.json", "r") as f:
    story_data = json.load(f)

print("🔬 Tracking Analysis:\n")

# Pipeline-level tracking
print("📊 Pipeline-Level Tracking:")
pipeline_keys = [
    "pipeline_name",
    "duration",
    "total_nodes",
    "completed_nodes",
    "success_rate",
    "total_rows_processed",
]
for key in pipeline_keys:
    if key in story_data:
        print(f"   ✓ {key}: {story_data[key]}")

# Node-level tracking
print("\n📋 Node-Level Tracking (per node):")
if story_data["nodes"]:
    sample_node = story_data["nodes"][0]
    node_keys = [
        "node_name",
        "operation",
        "status",
        "duration",
        "rows_in",
        "rows_out",
        "rows_change",
        "rows_change_pct",
        "schema_in",
        "schema_out",
        "columns_added",
        "columns_removed",
    ]
    for key in node_keys:
        if key in sample_node:
            print(f"   ✓ {key}")

# Timestamp tracking
print("\n🕐 Timestamp Tracking:")
timestamp_keys = ["started_at", "completed_at"]
for key in timestamp_keys:
    if key in story_data:
        print(f"   ✓ {key}: {story_data[key][:19]}")

# Context tracking
print("\n🏢 Context Tracking:")
context_keys = ["project", "plant", "asset", "pipeline_layer"]
for key in context_keys:
    if key in story_data and story_data[key]:
        print(f"   ✓ {key}: {story_data[key]}")

print("\n✅ Complete metadata tracking enabled!")

## ✅ Self-Check

In [ ]:
import time

check_start = time.time()

try:
    # Check all 4 artifact files exist
    required_files = ["story.html", "story.md", "story.json", "pipeline_metadata.json"]

    for filename in required_files:
        file_path = artifacts_dir / filename
        assert file_path.exists(), f"{filename} not found"

    print("✅ All 4 artifact files exist")

    # Validate story.json structure
    with open(artifacts_dir / "story.json") as f:
        story = json.load(f)

    assert "pipeline_name" in story, "Missing pipeline_name"
    assert "duration" in story, "Missing duration"
    assert "nodes" in story, "Missing nodes"
    assert story["pipeline_name"] == "demo_data_transformation", "Wrong pipeline name"

    print("✅ story.json has valid structure")
    print(f"   - pipeline_name: {story['pipeline_name']}")
    print(f"   - duration: {story['duration']:.4f}s")
    print(f"   - nodes: {len(story['nodes'])}")

    # Validate node count
    assert len(story["nodes"]) >= 3, f"Expected at least 3 nodes, got {len(story['nodes'])}"
    print("✅ Pipeline has at least 3 nodes")

    # Validate HTML contains expected content
    with open(artifacts_dir / "story.html", "r", encoding="utf-8") as f:
        html_content = f.read()

    html_keywords = ["Pipeline", "story", "metadata", "demo_data_transformation"]
    found_keyword = any(keyword.lower() in html_content.lower() for keyword in html_keywords)
    assert found_keyword, "HTML missing expected keywords"
    print("✅ HTML contains pipeline story content")

    # Validate pipeline metadata
    with open(artifacts_dir / "pipeline_metadata.json") as f:
        metadata = json.load(f)

    assert metadata["total_nodes"] >= 3, "Metadata missing nodes"
    assert metadata["completed_nodes"] >= 3, "Not all nodes completed"
    print("✅ Pipeline metadata complete")
    print(f"   - total_nodes: {metadata['total_nodes']}")
    print(f"   - completed_nodes: {metadata['completed_nodes']}")

    # Check node metadata fields
    node = story["nodes"][0]
    required_fields = ["node_name", "operation", "status", "duration", "rows_in", "rows_out"]
    for field in required_fields:
        assert field in node, f"Node missing field: {field}"
    print("✅ Node metadata has all required fields")

    # Check runtime
    elapsed = time.time() - check_start
    # Note: Total runtime includes the pipeline execution time above
    assert pipeline_metadata.duration < 30, (
        f"Pipeline runtime {pipeline_metadata.duration:.1f}s exceeds 30s budget"
    )

    print("\n🎉 Walkthrough verified successfully!")
    print(f"⏱️  Check time: {elapsed:.2f}s")
    print(f"⏱️  Pipeline time: {pipeline_metadata.duration:.2f}s")
    print(f"📊 Nodes executed: {metadata['total_nodes']}")
    print(f"📁 Artifacts created: {len(required_files)}")
    print("✅ All checks passed!")

except AssertionError as e:
    print(f"❌ Walkthrough failed: {e}")
    raise
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

## 🧠 Reflection

### What You Learned

1. **Automatic Metadata Tracking**: ODIBI captures comprehensive execution metadata without manual instrumentation
2. **Multi-Format Stories**: Same metadata can be rendered as HTML (interactive), Markdown (readable), or JSON (machine-readable)
3. **Rich Metrics**: Tracks performance, data quality, schema evolution, and execution status
4. **Hierarchical Structure**: Pipeline-level and node-level metadata provide both overview and details

### Where This Fits in ODIBI

```
Pipeline Execution Flow:
YAML → Parse → Execute Nodes → Capture Metadata → Generate Stories
                        ↓              ↓              ↓
                   NodeResult  →  Metadata  →  HTML/MD/JSON
                                      ↑
                         This notebook explained this!
```

Story generation is **essential for observability and auditability**. It creates automatic documentation and provides insights into:
- What happened during execution
- How long each step took
- What data transformations occurred
- Where failures occurred and why

### Key Insights

- **Zero-config tracking**: Metadata collection happens automatically during pipeline execution
- **Multiple audiences**: HTML for humans, JSON for machines, Markdown for documentation
- **Complete audit trail**: Every pipeline run creates a permanent, shareable record
- **Schema evolution**: Track how data structure changes through the pipeline
- **Performance insights**: Duration tracking helps identify bottlenecks

### Story Renderer Use Cases

- **HTMLStoryRenderer**: Interactive dashboards, executive reports, team reviews
- **MarkdownStoryRenderer**: Documentation, README files, GitHub wikis
- **JSONStoryRenderer**: API responses, monitoring systems, data warehouses

---

## ⏭ Next Steps

**Continue exploring:**
- Learn about custom story themes for branded reports
- Explore StoryGenerator for automated story creation during pipeline runs
- Integrate story metadata with monitoring and alerting systems

**Deep dive:**
- Read `odibi/story/metadata.py` - Metadata data classes and tracking logic
- Read `odibi/story/renderers.py` - Story rendering implementations
- Read `odibi/story/generator.py` - Automatic story generation
- Explore `odibi/story/templates/` - HTML template customization